### Assigning US Census Demographics for Each City

---

In [1]:
import pandas as pd
import numpy as np
import array

In [2]:
city_pumas = pd.read_csv('city_pumas.csv')

In [3]:
len(city_pumas)

113

In [4]:
city_pumas.head(1)

,City,PUMAs
0,Aliso Viejo,[5903]


In [5]:
type(city_pumas['PUMAs'][0])

str

In [6]:
# Converting lists in str format back to lists containing ints
city_pumas['PUMAs'] = city_pumas['PUMAs'].apply(lambda x:x[1:-1].split(',')).apply(lambda x:[int(s) for s in x])

In [7]:
type(city_pumas['PUMAs'][0])

list

---

### Cleaning Census Demographic Data

In [8]:
# This Census data was extracted from the following source, by manually choosing the variables of interest: https://usa.ipums.org/usa-action/variables/group
census_df = pd.read_csv('usa_00145.csv')

In [9]:
census_df.head(1)

,YEAR,STATEFIP,PUMA,PERWT,SEX,AGE,RACE,RACED,HISPAN,HISPAND,INCTOT
0,2017,1,2500,206,1,73,2,200,0,0,10000


In [10]:
# County indicated by first two digits of PUMA
census_df['COUNTY'] = census_df.PUMA.astype(str).apply(lambda x:x[:2]).astype(int)

In [11]:
census_df.head(1)

,YEAR,STATEFIP,PUMA,PERWT,SEX,AGE,RACE,RACED,HISPAN,HISPAND,INCTOT,COUNTY
0,2017,1,2500,206,1,73,2,200,0,0,10000,25


In [12]:
# Keeping only records of individuals residing in LA & Orange counties
census_df = census_df[(census_df.COUNTY.isin([37, 59])) & (census_df.STATEFIP == 6)]

In [13]:
census_df = census_df.drop(['RACED', 'HISPAND'], axis='columns')

In [14]:
census_df.head(1)

,YEAR,STATEFIP,PUMA,PERWT,SEX,AGE,RACE,HISPAN,INCTOT,COUNTY
153288,2017,6,3727,94,1,47,1,0,87400,37


---

Cleaning Race Columns:

In [15]:
# Replacing numeric code w/ corresponding race; source: https://usa.ipums.org/usa-action/variables/RACE#codes_section
race = {
1:	'White',
2:	'Black/African American/Negro',
3:	'American Indian or Alaska Native',
4:	'Chinese',
5:	'Japanese',
6:	'Other Asian or Pacific Islander',
7:	'Other race, nec',
8:	'Two major races',
9:	'Three or more major races'}

census_df.RACE = census_df.RACE.map(race)

In [16]:
# Source: https://usa.ipums.org/usa-action/variables/HISPAN#codes_section
hispanic = {
0:	'Not Hispanic',
1:	'Mexican',
2:	'Puerto Rican',
3:	'Cuban',
4:	'Other',
9:	'Not Reported'}

census_df.HISPAN = census_df.HISPAN.map(hispanic)

In [17]:
census_df.RACE.value_counts(normalize=True)

White                               0.547109
Other race, nec                     0.172237
Other Asian or Pacific Islander     0.115565
Black/African American/Negro        0.056043
Chinese                             0.049994
Two major races                     0.035295
Japanese                            0.010825
American Indian or Alaska Native    0.007080
Three or more major races           0.005852
Name: RACE, dtype: float64

In [18]:
census_df.HISPAN.value_counts(normalize=True)

Not Hispanic    0.576840
Mexican         0.328540
Other           0.085379
Puerto Rican    0.005041
Cuban           0.004200
Name: HISPAN, dtype: float64

In [19]:
# Assigning 'Other' to American Indian due to small count; merging Chinese and Japanese with Asian; merging two/three + races and assigning 'Mixed'
races = {'American Indian or Alaska Native': 'Other', 'Black/African American/Negro': 'African American',
       'Other race, nec':'Other', 'White':'White', 'Other Asian or Pacific Islander':'Asian',
       'Two major races':'Mixed', 'Chinese':'Asian', 'Japanese':'Asian',
       'Three or more major races':'Mixed'}

census_df.RACE = census_df.RACE.map(races)

In [20]:
census_df.RACE.value_counts(normalize=True)

White               0.547109
Other               0.179317
Asian               0.176384
African American    0.056043
Mixed               0.041147
Name: RACE, dtype: float64

In [21]:
census_df.HISPAN.value_counts(normalize=True)

Not Hispanic    0.576840
Mexican         0.328540
Other           0.085379
Puerto Rican    0.005041
Cuban           0.004200
Name: HISPAN, dtype: float64

In [22]:
# Simplifying values to 'Hispanic' or 'Not Hispanic'
census_df.loc[census_df.HISPAN != 'Not Hispanic', 'HISPAN'] = 'Hispanic'

In [23]:
census_df.HISPAN.value_counts(normalize=True)

Not Hispanic    0.57684
Hispanic        0.42316
Name: HISPAN, dtype: float64

In [24]:
# Reflecting 'Hispanic' in RACE column (which excludes Hispanic race- presumably categorized as 'white' or 'other')
census_df.loc[census_df.HISPAN != 'Not Hispanic', 'RACE'] = 'Hispanic'

In [25]:
census_df.RACE.value_counts(normalize=True)

Hispanic            0.423160
White               0.319731
Asian               0.173617
African American    0.053102
Mixed               0.025819
Other               0.004571
Name: RACE, dtype: float64

In [26]:
census_df = census_df.drop('HISPAN', axis='columns')

In [27]:
census_df.head(3)

,YEAR,STATEFIP,PUMA,PERWT,SEX,AGE,RACE,INCTOT,COUNTY
153288,2017,6,3727,94,1,47,White,87400,37
153289,2017,6,3727,123,2,41,White,70000,37
153305,2017,6,3732,61,2,41,Hispanic,40000,37


---

#### Extracting Census Data for Each City

In [28]:
census_df.head(1)

,YEAR,STATEFIP,PUMA,PERWT,SEX,AGE,RACE,INCTOT,COUNTY
153288,2017,6,3727,94,1,47,White,87400,37


In [29]:
city_pumas.head(1)

,City,PUMAs
0,Aliso Viejo,[5903]


In [30]:
cities = []
pumas_called = []
all_incomes = array.array('i')
med_ages = []
med_incomes = []
male_pers = []
hispanic_pers = []
incomes_dict = {}

for city in city_pumas.City:
    pumas = city_pumas.PUMAs[city_pumas.City == city].iloc[0]
    df = census_df[census_df.PUMA.isin(pumas)]
    ages = array.array('i')
    incomes = array.array('i')
    
    for i, row in df.iterrows():
        ages.extend(np.full(row.PERWT, row.AGE))  # Description of PERWT: https://usa.ipums.org/usa-action/variables/PERWT#description_section
        if row.INCTOT != 9999999:  # Code indicating n/a
            incomes.extend(np.full(row.PERWT, row.INCTOT))
        
    # Collecting all incomes w/o overlap (since some cities share the same PUMA)
    for puma in pumas:
        if puma in pumas_called:
            continue
        else:  # Individual PUMA incomes
            for i, row in census_df[census_df.PUMA == puma].iterrows():
                if row.INCTOT != 9999999:  
                    all_incomes.extend(np.full(row.PERWT, row.INCTOT))
                    # Cumulative census incomes
                    if row.INCTOT in incomes_dict:
                        incomes_dict[row.INCTOT] += row.PERWT
                    else:
                        incomes_dict[row.INCTOT] = row.PERWT
            pumas_called.append(puma)
            
    cities.append(city)
    med_ages.append(np.median(ages))
    med_incomes.append(np.median(incomes))
    male_pers.append(round(df.PERWT[df.SEX == 1].sum() / df.PERWT.sum(), 2)) 
    hispanic_pers.append(round(df.PERWT[df.RACE == 'Hispanic'].sum() / df.PERWT.sum(), 2))

In [127]:
len(cities)

113

---

Comparing Median Incomes:

In [31]:
np.median(all_incomes)  # All recorded incomes in the entire df, weighted by PERWT (which gives the most accurate figure)

21800.0

In [32]:
np.median(med_incomes)  # Median of all city median incomes

24000.0

*Conclusion: the median in med_incomes is close enough to all_incomes to rely on

---

In [33]:
demographics_df = pd.DataFrame({'City':cities, 'Median_Age':med_ages, 'Median_Income':med_incomes, 'Normalized_Income':(med_incomes-min(med_incomes)) / (max(med_incomes) - min(med_incomes)), 
                             'Male_Percent':male_pers, 'Hispanic_Percent':hispanic_pers})

In [34]:
demographics_df.Normalized_Income = demographics_df.Normalized_Income.apply(lambda x:round(x, 2))

In [35]:
demographics_df.head()

,City,Median_Age,Median_Income,Normalized_Income,Male_Percent,Hispanic_Percent
0,Aliso Viejo,46.0,40000.0,0.76,0.47,0.14
1,Anaheim,34.0,20000.0,0.22,0.51,0.55
2,Brea,39.0,31800.0,0.54,0.50,0.34
3,Buena Park,41.0,24000.0,0.33,0.49,0.28
4,Costa Mesa,36.0,30000.0,0.49,0.50,0.28


In [36]:
len(demographics_df)

113

In [37]:
demographics_df.to_csv('demographics_LA_OC.csv', index=False)

---